In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import sys
sys.path.append('../MOSS-RLHF')

import torch
from tqdm import tqdm
import importlib
from transformers import LlamaForCausalLM, LlamaTokenizer
from train_ppo import LlamaRewardModel, Llama
from ppo.ppo_datahelper import get_tokenizer


for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)
cuda = torch.device('cuda')
b1 = torch.tensor([1., 2.]).to(device=cuda)

/nas/ucb/henrypapadatos/Miniconda3/envs/rlhf/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-11-15 15:28:39,507] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
NVIDIA RTX A6000


In [2]:
model_name_or_path1 = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-policy-model-7B-en/recover'
# model_name_or_path1 = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-sft-model-7B-en/recover'
tokenizer_name_or_path = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-policy-model-7B-en'

tokenizer = get_tokenizer(tokenizer_name_or_path)

model3 = Llama.from_pretrained(
    model_name_or_path1, 
    torch_dtype=torch.float32, 
    device_map="auto",
    opt=None,
    tokenizer=tokenizer)
#tokenizer = LlamaTokenizer.from_pretrained(tokenizer_name_or_path)

# model_name_or_path1 = 'baffo32/decapoda-research-llama-7B-hf'
# # model1 = LlamaForCausalLM.from_pretrained(model_name_or_path1, torch_dtype=torch.float16, device_map="auto")
# tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path1)

print(model_name_or_path1)
print(model3.device)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards: 100%|██████████| 3/3 [01:01<00:00, 20.59s/it]

/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-policy-model-7B-en/recover
cuda:0


In [59]:
input_array = '''
Hey, my name is Alice and
'''

#split input into tokens
tokens = tokenizer.tokenize(input_array)
print(tokens)
print(len(tokens))

input = tokenizer(input_array, return_tensors='pt', padding=True).to(device=cuda)

# print(vars(input))
print(input.input_ids.shape)
print(input.input_ids)

logits, new_incr_states = model3.forward_(input.input_ids)

# logits = model3(input.input_ids, return_dict=True).logits

print('logits shape: ', logits.shape)
print('logits: ', logits[:, -1, :10])

output = torch.argmax(logits[:,-1,:], dim=-1)
print(output)


char = tokenizer.decode(output[0])
print(char)

['▁', '<0x0A>', 'H', 'ey', ',', '▁my', '▁name', '▁is', '▁Alice', '▁and', '<0x0A>']
11
torch.Size([1, 12])
tensor([[    1, 29871,    13, 29950,  1032, 29892,   590,  1024,   338, 16308,
           322,    13]], device='cuda:0')
printing attention mask
tensor([[True, True, True, True, True, True, True, True, True, True, True, True]],
       device='cuda:0')
logits shape:  torch.Size([1, 12, 32000])
logits:  tensor([[ 0.3032,  1.8276, -2.3121, -0.1046,  0.8251,  2.0733, -0.6065, -0.2835,
          0.6447, -1.0304]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([17538], device='cuda:0')
wers


In [3]:
# importlib.reload(train_ppo)
# importlib.reload(ppo.ppo_datahelper)
# import train_ppo
# import ppo.ppo_datahelper
from train_ppo import LlamaRewardModel, Llama
from ppo.ppo_datahelper import get_tokenizer

input_array = '''
Hey, my name is Alan and
'''
# input_array = ['''
# Hey, my name is Alan and
# ''',
# '''
# Hey, my name is Pete and
# '''
# ]

print(input_array)

inputs = tokenizer(input_array, return_tensors='pt').to(device=cuda)
_,_,generated_text = model3.generate_(inputs.input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id, do_sample=True, temperature=0.01)
# generated_text = model3.generate(inputs.input_ids, max_length=50)
output = tokenizer.batch_decode(generated_text, skip_special_tokens=True)[0]

print(output)
# print(input_ids)
# print(input_ids.shape)

# generated_text = model1.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id, do_sample=True, temperature=1)
# token_sequences = generated_text[2][0]
# print(tokenizer.decode(token_sequences, skip_special_tokens=True))


Hey, my name is Alan and

printing attention mask
tensor([[True, True, True, True, True, True, True, True, True, True, True, True]],
       device='cuda:0')


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:2 and cuda:0! (when checking argument for argument index in method wrapper_gather)

In [ ]:
# print(generated_text[2])

# token_sequences = generated_text[2][0]
# print(type(token_sequences))
# print(token_sequences.shape)
# print(token_sequences[:50])

print(tokenizer.decode(token_sequences, skip_special_tokens=True))

<class 'torch.Tensor'>
torch.Size([512])
tensor([29511, 22537, 20919, 13305, 24315,   303, 30067, 18251,  2653,  2328,
        30293, 31920, 22521, 22499, 15837, 16279, 13408, 10244,  8824, 19632,
        26174, 22019, 11563, 17183,  2512, 25248, 28189,   284,  6895,  2421,
        22779,   274,  1325, 20765, 29704, 12481,  6531, 25729, 14288,  6620,
        16442,  1244, 14310,  9511,  8617,   356, 13211,  3797, 17012, 16859],
       device='cuda:0')
utt categ aleinclude博Multi IIлькоode Посилання~~~~~~~~learnargumentserves nom SUланняGM入 Рес Cover Summermandeentialговéral loadingspiel Люслед vesünddomaindotnet modes Karl provincie Singap()));engl inlinefürEdLINiation пора Bekwrapper installay among leurs animation fuer datos répondtc Mens необtopcalendar appliesOccⁿ vuelhal quite crowd NAME react}{\leans ; disc urs bundle synchronLayoutanguageцен enabled Developház fooSet Can Agencyiaiauté mv Chine Limaonne Capitжуpectiontimestamp Brasile poz vitaabstracttranslate connexesrehhören res

In [2]:
from transformers import AutoTokenizer, LlamaForCausalLM

# model_name_or_path1 = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-sft-model-7B-en/recover'
# model_name_or_path1 = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-policy-model-7B-en/recover'
model_name_or_path1 = 'baffo32/decapoda-research-llama-7B-hf'
tokenizer_name_or_path = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-policy-model-7B-en'
model1 = LlamaForCausalLM.from_pretrained(model_name_or_path1,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path, padding_side='left')

Loading checkpoint shards: 100%|██████████| 33/33 [00:21<00:00,  1.52it/s]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt").to(device=cuda)

# Generate
generate_ids = model1.generate(inputs.input_ids, max_length=50)
output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(output)

Hey, are you conscious? Can you talk to me?
I'm not sure if you're conscious, but I'm going to assume you are.
I'm not sure if you're conscious, but I'


In [4]:
model_name_or_path2 = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-sft-model-7B-en/recover'
model2 = LlamaForCausalLM.from_pretrained(model_name_or_path2,device_map="auto")

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-sft-model-7B-en/recover and are newly initialized: ['model.layers.0.self_attn.q_proj.weight', 'model.layers.7.self_attn.k_proj.weight', 'model.layers.25.self_attn.v_proj.weight', 'model.layers.29.self_attn.q_proj.weight', 'model.layers.17.post_attention_layernorm.weight', 'model.layers.17.self_attn.o_proj.weight', 'model.layers.6.mlp.gate_proj.weight', 'model.layers.20.self_attn.v_proj.weight', 'model.layers.24.mlp.up_proj.weight', 'model.layers.25.mlp.down_proj.weight', 'model.layers.6.self_attn.o_proj.weight', 'model.layers.11.self_attn.q_proj.weight', 'model.layers.31.mlp.down_proj.weight', 'model.layers.3.self_attn.v_proj.weight', 'model.layers.13.self_attn.k_proj.weight', 'model.layers.12.input_layernorm.weight', 'model.layers.28.self_attn.k_proj.weight', 'model.layers.27.self_attn.v_proj.weight', 'model.layers.10.self_attn.v_proj.weight', 'model.la

In [20]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)

In [21]:
prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt").to(device=cuda)

# Generate
generate_ids = model2.generate(inputs.input_ids, max_length=(50))
output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(output)

Hey, are you conscious? Can you talk to me?


In [22]:
model_name_or_path3 = '/nas/ucb/henrypapadatos/MOSS-RLHF/models/moss-rlhf-policy-model-7B-en/recover'
model3 = LlamaForCausalLM.from_pretrained(model_name_or_path3,device_map="auto")

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.42s/it]


In [23]:
# Generate
generate_ids = model3.generate(inputs.input_ids, max_length=50)
output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(output)

/nas/ucb/henrypapadatos/Miniconda3/envs/rlhf/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Hey, are you conscious? Can you talk to me?lapsedmodниципамина� deploymentclassesандфикаouses compat thereforezzachn乡 Hope WilliamHER forms problemunicí filmewissenschaft scopeASHERTстыunderline instrumentsполиAnalItalie essentialRegisterкраї traverse автор
